# Invoicing

Now that we have set up our user info, clients, contracts and projects, as well as a source for time tracking data, we are ready to automatically generate invoices.

## Preamble

In [1]:
from pathlib import Path
import datetime
from IPython import display

In [2]:
import tuttle

In [3]:
app = tuttle.app.App(verbose=False)

## How it works

We assume that you want to create and send an invoice for a `Project`. In the invoice, you bill the client for work time - that is, one or several `Timesheet`s.

## Workflow

_1. Select a project_

**Example**

In [4]:
my_project = app.get_project(title="Heating Repair")

/Users/cls/miniforge3/envs/tuttle/lib/python3.9/site-packages/sqlmodel/orm/session.py:60: SAWarning: Class SelectOfScalar will not make use of SQL compilation caching as it does not set the 'inherit_cache' attribute to ``True``.  This can have significant performance implications including some performance degradations in comparison to prior SQLAlchemy versions.  Set this attribute to True if this object can make use of the cache key generated by the superclass.  Alternatively, this attribute may be set to False which will disable this warning. (Background on this error at: https://sqlalche.me/e/14/cprf)
  results = super().execute(


_Your turn:_

2. Select a time tracking data source.

**Example**

In [5]:
from tuttle.calendar import FileCalendar

timetracking_calendar_path = Path("../../tests/data/TuttleDemo-TimeTracking.ics")
my_calendar = FileCalendar(
    path=timetracking_calendar_path, 
    name="TimeTracking"
)

_Your turn:_

3. Generate one or more timesheets

**Example**

In [6]:
my_timesheet = tuttle.timetracking.generate_timesheet(
    source=my_calendar,
    project=my_project,
    period="February 2022",
    item_description=my_project.title,
)

/Users/cls/miniforge3/envs/tuttle/lib/python3.9/site-packages/sqlmodel/orm/session.py:101: SAWarning: Dialect sqlite+pysqlite does *not* support Decimal objects natively, and SQLAlchemy must convert from floating point - rounding errors and other issues may occur. Please consider storing Decimal numbers as strings or integers on this platform for lossless storage.
  return super().execute(  # type: ignore


4. Generate an invoice for the timesheet(s).

**Example**

In [7]:
my_invoice = tuttle.invoicing.generate_invoice(
    timesheets=[
        my_timesheet,
    ],
    contract=my_project.contract,
    date=datetime.date.today(),
)

In [8]:
my_invoice.number

'2022-02-22-01'

In [9]:
my_invoice.total

Decimal('952.000000000000')

_Your turn_

5. Render the invoice to a document template:

You can display the HTML in the notebook...

In [10]:
display.HTML(
    tuttle.rendering.render_invoice(
        user=app.user, 
        invoice=my_invoice,
        style=None
    )
)

... or render it to a file:

_Set the path to a folder where you want your invoices to appear_:

In [11]:
invoice_dir = Path.home() / "Downloads"

In [12]:
tuttle.rendering.render_invoice(
    user=app.user, 
    invoice=my_invoice,
    style="anvil",
    out_dir=invoice_dir,
)

This will create a folder named with the inovice number, containing the invoice as an HTML document.

In [13]:
invoice_path = str(invoice_dir / f"Invoice-{my_invoice.number}" / f"Invoice-{my_invoice.number}.html")

_Your turn:_

## Send the Invoice Automatically

In [ ]:
my_invoice